<a href="https://colab.research.google.com/github/allnes/pcam_train/blob/master/pcam_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
cd  '/content/drive/My Drive/DL_PCAM'

/content/drive/My Drive/DL_PCAM


In [10]:
%ls

camelyonpatch_level_2_split_test_meta.csv
camelyonpatch_level_2_split_test_x.h5
camelyonpatch_level_2_split_test_x.h5.gz
camelyonpatch_level_2_split_test_y.h5
camelyonpatch_level_2_split_test_y.h5.gz
camelyonpatch_level_2_split_train_meta.csv
camelyonpatch_level_2_split_train_x.h5
camelyonpatch_level_2_split_train_x.h5.gz
camelyonpatch_level_2_split_train_y.h5
camelyonpatch_level_2_split_train_y.h5.gz
camelyonpatch_level_2_split_valid_meta.csv
camelyonpatch_level_2_split_valid_x.h5
camelyonpatch_level_2_split_valid_x.h5.gz
camelyonpatch_level_2_split_valid_y.h5
camelyonpatch_level_2_split_valid_y.h5.gz
__pycache__/


In [0]:
import gzip
import shutil
def uzip_file(file_name, origin_name):
  with gzip.open(file_name, 'rb') as f_in:
      with open(origin_name, 'wb') as f_out:
          shutil.copyfileobj(f_in, f_out)


In [0]:
train_x_name = 'camelyonpatch_level_2_split_train_x.h5'
train_y_name = 'camelyonpatch_level_2_split_train_y.h5'

test_x_name = 'camelyonpatch_level_2_split_test_x.h5'
test_y_name = 'camelyonpatch_level_2_split_test_y.h5'

valid_x_name = 'camelyonpatch_level_2_split_valid_x.h5'
valid_y_name = 'camelyonpatch_level_2_split_valid_y.h5'

meta_train_name = 'camelyonpatch_level_2_split_train_meta.csv'
meta_valid_name = 'camelyonpatch_level_2_split_valid_meta.csv'
meta_test_name  = 'camelyonpatch_level_2_split_test_meta.csv'

# uzip_file(train_x_name + '.gz', train_x_name)
# uzip_file(train_y_name + '.gz', train_y_name)

# uzip_file(test_x_name + '.gz', test_x_name)
# uzip_file(test_y_name + '.gz', test_y_name)

# uzip_file(valid_x_name + '.gz', valid_x_name)
# uzip_file(valid_y_name + '.gz', valid_y_name)


In [0]:
from keras.utils import HDF5Matrix

x_train = HDF5Matrix(train_x_name, 'x')
y_train = HDF5Matrix(train_y_name, 'y')

x_valid = HDF5Matrix(valid_x_name, 'x')
y_valid = HDF5Matrix(valid_y_name, 'y')

x_test = HDF5Matrix(test_x_name, 'x')
y_test = HDF5Matrix(test_y_name, 'y')

meta_train = pd.read_csv(meta_train_name)
meta_valid = pd.read_csv(meta_valid_name)
meta_test = pd.read_csv(meta_test_name)

Using TensorFlow backend.


In [0]:
print(len(x_train))
print(type(x_train))
print(type(x_train[0]))
print(x_train[0].shape)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape 
from keras.regularizers import l2

In [0]:
full_size = x_train[0].shape[0] * x_train[0].shape[1] * x_train[0].shape[2]


In [0]:
model = Sequential([
    Reshape((1, 1, full_size,), input_shape=x_train[0].shape),
    Dense(13824), Activation('relu'),
    Dense(6912),  Activation('relu'),
    Dense(3456),  Activation('relu'),
    Dense(1728),  Activation('relu'),
    Dense(864),   Activation('relu'),
    Dense(432),   Activation('relu'),
    Dense(216),   Activation('relu'),
    Dense(108),   Activation('relu'),
    Dense(54),    Activation('relu'),
    Dense(27),    Activation('relu'),
    Dense(9),     Activation('relu'),
    Dense(3),     Activation('relu'),
    Dense(1),     Activation('softmax'),
])
model.summary()

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [0]:
model.fit(x_train, y_train, epochs=10, batch_size=16384, shuffle='batch')

In [0]:
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy: %.2f' % (accuracy * 100))